In [1]:
import requests
import zipfile
import os

import pandas as pd
import numpy as np
import sqlalchemy as sa
import dotenv

In [2]:
dotenv.load_dotenv()

True

In [3]:
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_DATABASE = os.getenv("PG_DATABASE")

In [4]:
response = requests.get(
    "https://www.kaggle.com/api/v1/datasets/download/atulanandjha/temperature-readings-iot-devices"
)
if response.status_code == 200:
    with open("data.zip", "wb") as fout:
        fout.write(response.content)

In [5]:
def unzip_file(zip_file_path, extract_to_path):
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(extract_to_path)

In [6]:
zip_file_path = "data.zip"
extract_to_path = "data"

os.makedirs(extract_to_path, exist_ok=True)

unzip_file(zip_file_path, extract_to_path)

In [7]:
df = pd.read_csv("data/IOT-temp.csv")

In [8]:
df.head()

,id,room_id/id,noted_date,temp,out/in
0,__export__.temp_log_196134_bd201015,Room Admin,08-12-2018 09:30,29,In
1,__export__.temp_log_196131_7bca51bc,Room Admin,08-12-2018 09:30,29,In
2,__export__.temp_log_196127_522915e3,Room Admin,08-12-2018 09:29,41,Out
3,__export__.temp_log_196128_be0919cf,Room Admin,08-12-2018 09:29,41,Out
4,__export__.temp_log_196126_d30b72fb,Room Admin,08-12-2018 09:29,31,In


# HW_02

In [9]:
df = df[df["out/in"] == "In"]
df["noted_date"] = pd.to_datetime(df["noted_date"], format="%d-%m-%Y %H:%M")
df = df[["noted_date", "temp"]]

In [10]:
df.head()

,noted_date,temp
0,2018-12-08 09:30:00,29
1,2018-12-08 09:30:00,29
4,2018-12-08 09:29:00,31
5,2018-12-08 09:29:00,31
6,2018-12-08 09:28:00,29


In [11]:
df[
    (df["temp"] >= np.percentile(df["temp"], 5))
    & (df["temp"] <= np.percentile(df["temp"], 95))
]

,noted_date,temp
0,2018-12-08 09:30:00,29
1,2018-12-08 09:30:00,29
4,2018-12-08 09:29:00,31
5,2018-12-08 09:29:00,31
6,2018-12-08 09:28:00,29
...,...,...
97601,2018-07-28 07:07:00,31
97602,2018-07-28 07:07:00,31
97603,2018-07-28 07:06:00,31
97604,2018-07-28 07:06:00,31


# HW_03

In [12]:
PG_URL = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"

In [13]:
engine = sa.create_engine(PG_URL)

In [14]:
def upload(df, table_name, engine):
    df.to_sql(table_name, engine, if_exists="replace", index=False)

In [15]:
def increment_upload(df, table_name, engine):
    df.to_sql(table_name, engine, if_exists="append", index=False)

In [16]:
upload(df, "temperature", engine)